ref1: [here](https://colab.research.google.com/github/Hvass-Labs/TensorFlow-Tutorials/blob/master/21_Machine_Translation.ipynb#scrollTo=KXDkj10wHXKK)

ref2 from TF: [here](https://www.tensorflow.org/text/tutorials/nmt_with_attention)

In [ ]:
import string
import numpy as np
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Model
from keras.layers import LSTM, Input, TimeDistributed, Dense,Activation, RepeatVector, Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

In [ ]:
import pathlib

path_to_zip = tf.keras.utils.get_file(
    'spa-eng.zip', origin='http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip',
    extract=True)

path_to_data = pathlib.Path(path_to_zip).parent/'spa-eng/spa.txt'

2638744/2638744 [==============================] - 1s 0us/step


In [ ]:
translation_file = open(path_to_data,"r", encoding='utf-8')
raw_data = translation_file.read()
translation_file.close()

In [ ]:
raw_data = raw_data.split('\n')
pairs = [sentence.split('\t') for sentence in raw_data]
pairs = pairs[1000:20000]

In [ ]:
def clean_sentence(sentence):
  # Lower case the sentence
  lower_case_sent = sentence.lower() # Strip punctuation
  string_punctuation = string.punctuation + "¡" + '¿'
  clean_sentence = lower_case_sent.translate(str.maketrans('', '', string_punctuation))
  return clean_sentence

def tokenize(sentences): # Create tokenizer
  text_tokenizer = Tokenizer() # Fit texts
  text_tokenizer.fit_on_texts(sentences)
  return text_tokenizer.texts_to_sequences(sentences), text_tokenizer

english_sentences = [clean_sentence(pair[0]) for pair in pairs]
spanish_sentences = [clean_sentence(pair[1]) for pair in pairs]

In [ ]:
# Tokenize words
spa_text_tokenized, spa_text_tokenizer = tokenize(spanish_sentences)
eng_text_tokenized, eng_text_tokenizer = tokenize(english_sentences)

print('Maximum length spanish sentence: {}'.format(len(max(spa_text_tokenized,key=len))))
print('Maximum length english sentence: {}'.format(len(max(eng_text_tokenized,key=len))))

Maximum length spanish sentence: 12
Maximum length english sentence: 6


In [ ]:
# Check language length
spanish_vocab = len(spa_text_tokenizer.word_index) + 1
english_vocab = len(eng_text_tokenizer.word_index) + 1
print("Spanish vocabulary is of {} unique words".format(spanish_vocab))
print("English vocabulary is of {} unique words".format(english_vocab))

Spanish vocabulary is of 7239 unique words
English vocabulary is of 3804 unique words


In [ ]:
max_spanish_len = int(len(max(spa_text_tokenized,key=len)))
max_english_len = int(len(max(eng_text_tokenized,key=len)))
spa_pad_sentence = pad_sequences(spa_text_tokenized, max_spanish_len, padding = "post")
eng_pad_sentence = pad_sequences(eng_text_tokenized, max_english_len, padding = "post")

In [ ]:
# Reshape data
spa_pad_sentence = spa_pad_sentence.reshape(*spa_pad_sentence.shape, 1)
eng_pad_sentence = eng_pad_sentence.reshape(*eng_pad_sentence.shape, 1)

In [ ]:
# input_sequence = Input(shape=(max_spanish_len,))
# embedding = Embedding(input_dim=spanish_vocab, output_dim=128,)(input_sequence)

In [ ]:
# input_sequence = Input(shape=(max_spanish_len,))
# embedding = Embedding(input_dim=spanish_vocab, output_dim=128,)(input_sequence)
# encoder = LSTM(64, return_sequences=False)(embedding)

In [ ]:
# input_sequence = Input(shape=(max_spanish_len,))
# embedding = Embedding(input_dim=spanish_vocab, output_dim=128,)(input_sequence)
# encoder = LSTM(64, return_sequences=False)(embedding)
# r_vec = RepeatVector(max_english_len)(encoder)

In [ ]:
# input_sequence = Input(shape=(max_spanish_len,))
# embedding = Embedding(input_dim=spanish_vocab, output_dim=128,)(input_sequence)
# encoder = LSTM(64, return_sequences=False)(embedding)
# r_vec = RepeatVector(max_english_len)(encoder)
# decoder = LSTM(64, return_sequences=True, dropout=0.2)(r_vec)

In [ ]:
input_sequence = Input(shape=(max_spanish_len,))
embedding = Embedding(input_dim=spanish_vocab, output_dim=128,)(input_sequence)
encoder = LSTM(64, return_sequences=False)(embedding)
r_vec = RepeatVector(max_english_len)(encoder)
decoder = LSTM(64, return_sequences=True, dropout=0.2)(r_vec)
logits = TimeDistributed(Dense(english_vocab))(decoder)
enc_dec_model = Model(input_sequence, Activation('softmax')(logits))
enc_dec_model.compile(loss=sparse_categorical_crossentropy, optimizer=Adam(1e-3), metrics=['accuracy'])
enc_dec_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 12)]              0         
                                                                 
 embedding (Embedding)       (None, 12, 128)           926592    
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 repeat_vector (RepeatVecto  (None, 6, 64)             0         
 r)                                                              
                                                                 
 lstm_1 (LSTM)               (None, 6, 64)             33024     
                                                                 
 time_distributed (TimeDist  (None, 6, 3804)           247260    
 ributed)                                                    

In [ ]:
enc_dec_model.fit(spa_pad_sentence, eng_pad_sentence, epochs=50)

Epoch 1/50
594/594 [==============================] - 22s 13ms/step - loss: 4.1397 - accuracy: 0.4329
Epoch 2/50
594/594 [==============================] - 5s 8ms/step - loss: 3.5077 - accuracy: 0.4647
Epoch 3/50
594/594 [==============================] - 5s 8ms/step - loss: 3.4407 - accuracy: 0.4648
Epoch 4/50
594/594 [==============================] - 4s 7ms/step - loss: 3.4119 - accuracy: 0.4648
Epoch 5/50
594/594 [==============================] - 4s 7ms/step - loss: 3.3954 - accuracy: 0.4648
Epoch 6/50
594/594 [==============================] - 5s 8ms/step - loss: 3.3840 - accuracy: 0.4647
Epoch 7/50
594/594 [==============================] - 4s 7ms/step - loss: 3.3485 - accuracy: 0.4683
Epoch 8/50
594/594 [==============================] - 4s 7ms/step - loss: 3.2676 - accuracy: 0.4738
Epoch 9/50
594/594 [==============================] - 5s 8ms/step - loss: 3.1798 - accuracy: 0.4803
Epoch 10/50
594/594 [==============================] - 4s 7ms/step - loss: 3.0158 - accuracy: 0.49

In [ ]:
def logits_to_sentence(logits, tokenizer):
  index_to_words = {idx: word for word, idx in tokenizer.word_index.items()}
  index_to_words[0] = '<empty>'
  return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

indexes = [1, 17]
for index in indexes:
  print("The english sentence is: {}".format(english_sentences[index]))
  print("The spanish sentence is: {}".format(spanish_sentences[index]))
  print('The predicted sentence is :')
  print(logits_to_sentence(enc_dec_model.predict(spa_pad_sentence[index:index+1])[0], eng_text_tokenizer))
  print()

The english sentence is: he is here
The spanish sentence is: él está aquí
The predicted sentence is :
1/1 [==============================] - 1s 618ms/step
he is here <empty> <empty> <empty>

The english sentence is: hes smart
The spanish sentence is: él es inteligente
The predicted sentence is :
1/1 [==============================] - 0s 16ms/step
hes intelligent <empty> <empty> <empty> <empty>

